<a href="https://colab.research.google.com/github/tanoManzo/mimic_trajectories/blob/dev/MIMIC_TRAJECTORY_embeddings_04_02_collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection, Embeddings and Attitude

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
notes_path = "/content/drive/MyDrive/NIH/Data/"

In [4]:
file_name = "ICUSTAYS.csv"
df_ward = pd.read_csv(notes_path+file_name)
subject_id_tsicu = df_ward[(df_ward['FIRST_CAREUNIT']=='TSICU') & (df_ward['LAST_CAREUNIT']=='TSICU')].SUBJECT_ID.unique()
print(f'Number of Subjects in TSICU: {len(subject_id_tsicu)}')
hosp_stay_id_tsicu = df_ward[(df_ward['FIRST_CAREUNIT']=='TSICU') & (df_ward['LAST_CAREUNIT']=='TSICU')].HADM_ID.unique()
print(f'Number of Hospital stays in TSICU: {len(hosp_stay_id_tsicu)}')

Number of Subjects in TSICU: 5418
Number of Hospital stays in TSICU: 5563


In [5]:
file_name = "DIAGNOSES_ICD.csv"
df_icd = pd.read_csv(notes_path+file_name)
df_icd_first = df_icd[df_icd['SEQ_NUM']==1]
df_icd_first_tsicu = df_icd_first[df_icd_first['SUBJECT_ID'].isin(subject_id_tsicu)]
print(f'Number of unique ICD 9 first sequence from TCICU subject: {df_icd_first_tsicu.ICD9_CODE.nunique()}')
icd_tsicu = df_icd_first_tsicu.ICD9_CODE.value_counts().nlargest(5)
print('Top 5 Popular ICD9 in the TSICU population with duplicates:')
print("code  #subjects")
icd_tsicu

Number of unique ICD 9 first sequence from TCICU subject: 1387
Top 5 Popular ICD9 in the TSICU population with duplicates:
code  #subjects


431      190
0389     164
85221    103
51881     81
430       79
Name: ICD9_CODE, dtype: int64

In [6]:
condition = []
for item in df_icd['ICD9_CODE'].values:
  
  if str(item).isdigit():
    item = float(item)
    if (item>799 and item<1000):
      condition.append(True)
    else: 
      condition.append(False)
  else:
      condition.append(False)

df_8xx = df_icd[condition]
df_8xx_tsicu = df_8xx[df_8xx['SUBJECT_ID'].isin(subject_id_tsicu)]
icd8xx_subjects_ids = df_8xx_tsicu['SUBJECT_ID'].unique()
icd8xx_hosp_ids = df_8xx_tsicu['HADM_ID'].unique()
print(f'Number of subjects with a first sequence ICD 9 code [800,900) : {len(icd8xx_subjects_ids)}')
print(f'Number of hospital stays with a first sequence ICD 9 code [800,900) : {len(icd8xx_hosp_ids)}')

Number of subjects with a first sequence ICD 9 code [800,900) : 272
Number of hospital stays with a first sequence ICD 9 code [800,900) : 290


In [7]:
print(df_8xx_tsicu['ICD9_CODE'].unique())

['920' '00845' '938' '936' '0940' '932' '0971' '0993' '937' '00841']


In [8]:
patients = "PATIENTS.csv"
df_patients = pd.read_csv(notes_path+patients)
df_patients_icd8xx_tsicu = df_patients[df_patients['SUBJECT_ID'].isin(icd8xx_subjects_ids)]
df_patients_icd8xx_tsicu.info()
print('Subjects Expired (1), Alive (0): ')
print(df_patients_icd8xx_tsicu['EXPIRE_FLAG'].value_counts())
print(df_patients_icd8xx_tsicu['GENDER'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 126 to 46370
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       272 non-null    int64 
 1   SUBJECT_ID   272 non-null    int64 
 2   GENDER       272 non-null    object
 3   DOB          272 non-null    object
 4   DOD          113 non-null    object
 5   DOD_HOSP     65 non-null     object
 6   DOD_SSN      96 non-null     object
 7   EXPIRE_FLAG  272 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 19.1+ KB
Subjects Expired (1), Alive (0): 
0    159
1    113
Name: EXPIRE_FLAG, dtype: int64
M    153
F    119
Name: GENDER, dtype: int64


In [9]:
df_gcs_total_subjects = df_patients_icd8xx_tsicu[df_patients_icd8xx_tsicu['SUBJECT_ID'].isin(icd8xx_subjects_ids)]
df_gcs_total_subjects['SUBJECT_ID'].nunique()

272

In [10]:
note_events = "NOTEEVENTS.csv"
df_note_events = pd.read_csv(notes_path+note_events)
df_note_events.info()

<ipython-input-10-8fd00f55f791>:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_note_events = pd.read_csv(notes_path+note_events)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


In [11]:
df_notes_total_hosp_stays = df_note_events[df_note_events['HADM_ID'].isin(hosp_stay_id_tsicu)] 
df_notes_total_hosp_stays.info()
df_notes_total_hosp_stays.sort_values(by=['HADM_ID','CHARTTIME'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166730 entries, 4 to 2066655
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ROW_ID       166730 non-null  int64  
 1   SUBJECT_ID   166730 non-null  int64  
 2   HADM_ID      166730 non-null  float64
 3   CHARTDATE    166730 non-null  object 
 4   CHARTTIME    150514 non-null  object 
 5   STORETIME    89105 non-null   object 
 6   CATEGORY     166730 non-null  object 
 7   DESCRIPTION  166730 non-null  object 
 8   CGID         89105 non-null   float64
 9   ISERROR      71 non-null      float64
 10  TEXT         166730 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 15.3+ MB


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
1125713,1146721,87977,100011.0,2177-08-29,2177-08-29 04:11:00,NaN,Radiology,CT HEAD W/O CONTRAST,NaN,NaN,[**2177-8-29**] 4:11 AM\n CT HEAD W/O CONTRAST...
1125714,1146722,87977,100011.0,2177-08-29,2177-08-29 04:12:00,NaN,Radiology,CT C-SPINE W/O CONTRAST,NaN,NaN,[**2177-8-29**] 4:12 AM\n CT C-SPINE W/O CONTR...
1125715,1146723,87977,100011.0,2177-08-29,2177-08-29 04:13:00,NaN,Radiology,CT PELVIS W/CONTRAST,NaN,NaN,[**2177-8-29**] 4:13 AM\n CT CHEST W/CONTRAST;...
1125355,1146724,87977,100011.0,2177-08-29,2177-08-29 04:26:00,NaN,Radiology,P TRAUMA #2 (AP CXR & PELVIS PORT) PORT,NaN,NaN,[**2177-8-29**] 4:26 AM\n TRAUMA #2 (AP CXR & ...
1125356,1146725,87977,100011.0,2177-08-29,2177-08-29 05:16:00,NaN,Radiology,R KNEE (2 VIEWS) RIGHT,NaN,NaN,[**2177-8-29**] 5:16 AM\n KNEE (2 VIEWS) RIGHT...
...,...,...,...,...,...,...,...,...,...,...,...
1014186,1018660,28549,199987.0,2175-05-27,2175-05-27 14:44:00,NaN,Radiology,CHEST (PA & LAT),NaN,NaN,[**2175-5-27**] 2:44 PM\n CHEST (PA & LAT) ...
3812,3727,28549,199987.0,2175-05-28,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2175-5-19**] ...
63666,63301,28549,199987.0,2175-05-22,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Left ve...
134685,127338,28549,199987.0,2175-05-24,NaN,NaN,ECG,Report,NaN,NaN,Sinus rhythm. Compared to the previous tracin...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [20]:
print(df_note_events['CATEGORY'].unique())

['Discharge summary' 'Echo' 'ECG' 'Nursing' 'Physician ' 'Rehab Services'
 'Case Management ' 'Respiratory ' 'Nutrition' 'General' 'Social Work'
 'Pharmacy' 'Consult' 'Radiology' 'Nursing/other']


In [73]:
import re
df_notes_icd8xx_tsicu_social = df_notes_total_hosp_stays.merge(df_patients[['SUBJECT_ID', 'EXPIRE_FLAG']], on=['SUBJECT_ID'], how='inner')
df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social.merge(df_8xx_tsicu[['SUBJECT_ID', 'ICD9_CODE','HADM_ID']], on=['SUBJECT_ID','HADM_ID'], how='inner')
df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[(df_notes_icd8xx_tsicu_social['CATEGORY']=='Nursing') | (df_notes_icd8xx_tsicu_social['CATEGORY']=='Nursing/other') | (df_notes_icd8xx_tsicu_social['CATEGORY']=='Social Work')]

# regex pattern
pattern = r'(?i)(social work note:|social work note-|social work note\n\n|social work note\n|social work:|social work-|social work\n\n|social work\n|social:|soc:|social-|social \n|social\n\n|social\n)(.*?)(?:\n|$)'

# grabs multiple results if the pattern shows more than once
df_notes_icd8xx_tsicu_social['SOCIAL'] = df_notes_icd8xx_tsicu_social['TEXT'].apply(lambda x: ' '.join([match[1] for match in re.findall(pattern, x.lower(), flags=re.DOTALL)]) if re.findall(pattern, x.lower(), flags=re.DOTALL) else '')

#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['TEXT'].str.lower().str.contains('social work', regex=True)] # grabs only "social work" examples
#df_notes_icd8xx_tsicu_social['SOCIAL'] = df_notes_icd8xx_tsicu_social['TEXT'].str.lower().str.extract(pattern, flags=re.DOTALL)[1]  # grabs only the first result of regex pattern

#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social.sort_values(by=['HADM_ID','CHARTTIME']) # for testing
#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['EXPIRE_FLAG'] == 1] # for testing
#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['HADM_ID'] == 108474] # for testing

df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social.sort_values(by='SOCIAL', key=lambda x: x.str.len(), ascending=False)
df_notes_icd8xx_tsicu_social.info()
df_notes_icd8xx_tsicu_social.head(200)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3698 entries, 4409 to 3828
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ROW_ID       3698 non-null   int64  
 1   SUBJECT_ID   3698 non-null   int64  
 2   HADM_ID      3698 non-null   float64
 3   CHARTDATE    3698 non-null   object 
 4   CHARTTIME    3698 non-null   object 
 5   STORETIME    3698 non-null   object 
 6   CATEGORY     3698 non-null   object 
 7   DESCRIPTION  3698 non-null   object 
 8   CGID         3698 non-null   float64
 9   ISERROR      0 non-null      float64
 10  TEXT         3698 non-null   object 
 11  EXPIRE_FLAG  3698 non-null   int64  
 12  ICD9_CODE    3698 non-null   object 
 13  SOCIAL       3698 non-null   object 
dtypes: float64(3), int64(3), object(8)
memory usage: 433.4+ KB


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,EXPIRE_FLAG,ICD9_CODE,SOCIAL
4409,1278472,1258,177293.0,2191-06-16,2191-06-16 20:34:00,2191-06-16 20:43:00,Nursing/other,Report,17159.0,NaN,"Social Work\nSW met with pt's mother, [**Name ...",1,920,"sw met with pt's mother, [**name (ni) **], in ..."
4730,1538749,21068,138137.0,2158-01-24,2158-01-24 05:05:00,2158-01-24 05:30:00,Nursing/other,Report,14562.0,NaN,NPN 1900-0700\nNEURO: Pt alert and interactive...,0,00845,[**name (ni) 15**] mother phoned in and expre...
1598,1435366,13249,164729.0,2198-01-18,2198-01-18 21:14:00,2198-01-18 21:22:00,Nursing/other,Report,15225.0,NaN,MICU NPN Addendum:\nSocial: Pt's daughter [**N...,1,00845,pt's daughter [**name (ni) **] called this ev...
3231,1321227,4579,114175.0,2193-06-19,2193-06-19 18:13:00,2193-06-19 18:26:00,Nursing/other,Report,17159.0,NaN,Social Work\nSW met with pt's wife today to in...,1,920,"sw met with pt's wife today to introduce sw, p..."
2234,1416235,11776,161707.0,2190-02-22,2190-02-22 14:07:00,2190-02-22 14:14:00,Nursing/other,Report,17665.0,NaN,SOCIAL WORK\nSW met with pts brothers [**Name ...,0,00845,sw met with pts brothers [**name (ni) 3057**] ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,1387018,9437,155329.0,2132-09-15,2132-09-15 06:41:00,2132-09-15 07:09:00,Nursing/other,Report,21202.0,NaN,Admission Note\n28 y.o.m. NKDA Full Code ...,0,920,fiance w/ pt all night. parents flew in from ...
2985,1660491,31133,102634.0,2106-03-20,2106-03-20 18:09:00,2106-03-20 18:19:00,Nursing/other,Report,15217.0,NaN,NPN TSICU 0700-1900\nReview of systems:\n\nneu...,1,00845,multiple family members into visit today. fam...
2686,1636320,28744,108474.0,2120-10-01,2120-10-01 05:11:00,2120-10-01 05:27:00,Nursing/other,Report,14151.0,NaN,"assessment as noted in carevue\n\nres: coarse,...",0,00845,family wa sin last night and talked to s/[**i...
3459,1671473,32287,197432.0,2177-03-16,2177-03-16 17:39:00,2177-03-16 17:57:00,Nursing/other,Report,17435.0,NaN,Nursing Progress Note\nNo significant events\n...,0,920,[**name (ni) 4**] sons and brother called tod...


In [ ]:
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_TSICU_TEXT_ONLY.csv'
df_notes_icd8xx_tsicu_social['TEXT'].to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

In [ ]:
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_TSICU_GCS_HADM.csv'
#df_notes_icd8xx_tsicu_gcs_scores.to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

In [ ]:
df_note_icd8xx_tsicu = df_note_events[df_note_events['SUBJECT_ID'].isin(icd8xx_subjects_ids)]
print(f'Number of total notes: {len(df_note_icd8xx_tsicu)}')
df_nursing_progress_note_icd8xx_tsicu = df_note_icd8xx_tsicu[df_note_icd8xx_tsicu['DESCRIPTION']=='Nursing Progress Note']
print(f'Number of Nursing Progress Notes: {len(df_nursing_progress_note_icd8xx_tsicu)}')
df_discharge_note_icd8xx_tsicu = df_note_icd8xx_tsicu[(df_note_icd8xx_tsicu['CATEGORY']=='Discharge summary') ]
print(f'Number of Discharge summary: {len(df_discharge_note_icd8xx_tsicu)}')

In [ ]:
cgid_icd8xx_tsicu = df_nursing_progress_note_icd8xx_tsicu['CGID'].unique()
#print(f'Number of caregivers: {len(cgid_maxicu_tsicu)}')

In [ ]:
caregivers = "CAREGIVERS.csv"
df_caregivers = pd.read_csv(notes_path+caregivers)
df_caregivers[df_caregivers['CGID'].isin(cgid_icd8xx_tsicu)].LABEL.value_counts()

In [ ]:
name_file= "ADMISSIONS.csv"
df_adm = pd.read_csv(notes_path+name_file)
df_adm_icd8xx_tsicu = df_adm[df_adm['SUBJECT_ID'].isin(cgid_icd8xx_tsicu)]
df_adm_icd8xx_tsicu

Report

In [ ]:
print(f'Number of Subjects in TSICU: {len(subject_id_tsicu)}')
print(f'Number of subjects with a first sequence ICD 9 code [800,900) : {len(icd8xx_subjects_ids)}')
print('')
print('Subjects Expired (1), Alive (0): ')
print(df_patients_icd8xx_tsicu['EXPIRE_FLAG'].value_counts())
print('')
print('Gender:')
print(df_patients_icd8xx_tsicu['GENDER'].value_counts())
print('')
print(f'Number of total notes: {len(df_note_icd8xx_tsicu)}')
print(f'Number of Nursing Progress Notes: {len(df_nursing_progress_note_icd8xx_tsicu)}')
print(f'Number of Discharge summary: {len(df_discharge_note_icd8xx_tsicu)}')
print(f'Number of caregivers: {len(cgid_icd8xx_tsicu)}')

# END


## **Data Preprocessing**

In [ ]:
df_note_events.info()

Grab tsicu notes, merge with ward and patients tables

In [ ]:
# get new type of notes (TSICU)
df_notes_TSICU = pd.DataFrame()
#df_notes_TSICU = df_notes.merge(df_note_events[['ROW_ID', 'CATEGORY']], on='ROW_ID')
df_notes_TSICU = df_note_events.merge(df_caregivers.drop('ROW_ID', axis=1), on='CGID')
#df_notes_TSICU = df_notes_TSICU.merge(df_ward.drop('ROW_ID', axis=1), on='SUBJECT_ID')
df_notes_TSICU = df_notes_TSICU.merge(df_patients.drop('ROW_ID', axis=1), on='SUBJECT_ID')
#df_notes_TSICU = df_notes_TSICU[(df_notes_TSICU['first_careunit'] == 'TSICU') & (df_notes_TSICU['last_careunit'] == 'TSICU')].drop_duplicates()
df_notes_TSICU.info()

In [ ]:
df_notes_TSICU.head(5)

Join with ICD

In [ ]:
icd = "DIAGNOSES_ICD.csv"
df_icd = pd.read_csv(notes_path+icd)
df_icd.columns = df_icd.columns.str.upper()
df_icd = df_icd.dropna(subset=['SEQ_NUM'])
df_icd['SEQ_NUM'] = df_icd['SEQ_NUM'].astype(int)
unique_values = df_icd['SEQ_NUM'].unique()
print("unique_values: ")
print(unique_values)
idx = df_icd.groupby(['HADM_ID', 'SUBJECT_ID'])['SEQ_NUM'].idxmin()
df_icd = df_icd.loc[idx]
df_icd.info()

In [ ]:
df_notes_TSICU = df_notes_TSICU.merge(df_icd[['HADM_ID','ICD9_CODE','SEQ_NUM', 'SUBJECT_ID']], on=['HADM_ID', 'SUBJECT_ID'])
df_notes_TSICU.info()

In [ ]:
df_notes_TSICU.head(20)

In [ ]:
grouping = df_notes_TSICU.groupby(['CGID','SUBJECT_ID']).size()
df_notes_TSICU['num_of_notes'] = df_notes_TSICU.set_index(['CGID','SUBJECT_ID']).index.map(grouping)
df_notes_TSICU_filtered = df_notes_TSICU[df_notes_TSICU['num_of_notes'] > 1]
df_notes_TSICU_filtered.info()

In [ ]:
df_notes_TSICU_filtered.head(10)

Filter by number of notes

In [ ]:
gb_notes = df_notes_TSICU_filtered.groupby(['ICD9_CODE'])
num_notes = gb_notes['TEXT'].count()

print(f'number of notes: min={num_notes.min()}, max={num_notes.max()}, mean={num_notes.mean()},  median={num_notes.median()}')
sns.boxplot(x=num_notes)
sns.displot(num_notes[((num_notes>2) & (num_notes<100))])

In [ ]:
df_notes_TSICU_filtered.info()

In [ ]:
icd9_grouping = df_notes_TSICU_filtered.groupby(['ICD9_CODE']).size()
df_notes_TSICU_filtered['num_of_conditions'] = df_notes_TSICU_filtered.set_index(['ICD9_CODE']).index.map(icd9_grouping)
#df_notes_TSICU_filtered = df_notes_TSICU_filtered[(df_notes_TSICU_filtered['num_of_conditions'] >= 15) & (df_notes_TSICU_filtered['num_of_conditions'] <= 1000)]
#df_notes_TSICU_filtered = df_notes_TSICU_filtered[(df_notes_TSICU_filtered['num_of_conditions'] >= 15) & (df_notes_TSICU_filtered['num_of_conditions'] <= 100)]
df_notes_TSICU_filtered.info()
print(icd9_grouping)

In [ ]:
df_notes_TSICU_dropped_cols = df_notes_TSICU_filtered.drop(['STORETIME','CHARTDATE','CHARTTIME','DOB','DOD','DOD_HOSP','DOD_SSN'], axis=1)
df_notes_TSICU_dropped_cols.columns

select emotional words then filter rows that dont have a minimum number of them

In [ ]:
def filter_for_emotional_words(df):
  selected_words = ['pain', 'family', 'stable', 'care', 'well', 'social', 'support', 'able', 'decreased', 'warm', 'unable', 'strong', 'intact', 'good', 'times', 'aware', 'eyes', 'tolerated', 'denies', 'tolerating', 'please', 'palpable']
  print("selected_words list length: ", len(selected_words))
  pattern = '|'.join(selected_words)
  new_df = df[df['TEXT'].str.contains(pattern, regex=True)].copy()
  #filtered_data = df_notes_TSICU[df_notes_TSICU['TEXT'].str.contains(pattern, regex=True)].copy()
  new_df['num_search_words'] = new_df['TEXT'].str.count(pattern)
  #new_df = new_df[new_df['num_search_words'] >= 0]
  new_df.info()
  return new_df

In [ ]:
#filtered_data = filter_for_emotional_words(df_notes_TSICU_dropped_cols)
filtered_data = df_notes_TSICU_dropped_cols

Random selection from sample set

In [ ]:
#random_filtered_data = filtered_data.sample(n=50000, random_state=42)
random_filtered_data = filtered_data

Remove similar sentences

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def remove_similar_notes(df):
  sentences = df['TEXT'].tolist()
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(sentences)
  similarity_threshold = 0.8

  similar_indices = []
  for i, sentence in enumerate(sentences):  # needs to be max 1000 entries
    if any(cosine_similarity(tfidf_matrix[i], tfidf_matrix[j])[0][0] > similarity_threshold for j in range(i)):  # very slow ~10 minutes for 1000 entries, exponential complexity (more than 5 hours if 6000 entries)
      similar_indices.append(i)
  index_labels = df.index[similar_indices]
  df.drop(index_labels, inplace=True)
  return df

Inverse Document Frequency

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_notes_icd8xx_tsicu_gcs_scores['TEXT'])

feature_names = vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.sum(axis=0).A1
word_score_dict = dict(zip(feature_names, tfidf_scores))

sorted_words = sorted(word_score_dict.items(), key=lambda x: x[1], reverse=True)

top_words = sorted_words[:50]

In [ ]:
print("Rank\tWord\t\t\tCount")
for rank, (word, score) in enumerate(top_words, start=1):
  print(f"{rank}\t{word.ljust(20)}\t{score}")

In [ ]:
#random_filtered_data = remove_similar_notes(random_filtered_data)
random_filtered_data.info()

Find matching conditions for both expired flags

In [ ]:
def split_set_half_expired(df):
  df['EXPIRE_FLAG'] = df['EXPIRE_FLAG'].astype(int)
  group_conditions = df.groupby(['ICD9_CODE', 'EXPIRE_FLAG'])
  def balance_conditions(g1,g2,size):
    size = min(len(g1), len(g2), size)
    return g1.sample(n=size, random_state=1), g2.sample(n=size, random_state=1)

  rows = []
  size = 50000

  for drg_desc, group in group_conditions:
    icd9, expire = drg_desc
    if (icd9, 1-expire) in group_conditions.groups:
      paired_group = group_conditions.get_group((icd9, 1-expire))
      group_sample1, group_sample2 = balance_conditions(group, paired_group, size)
      rows.append(group_sample1)
      rows.append(group_sample2)
      size -= len(group_sample1)
    if size <= 0:
      break

  new_df = pd.concat(rows)
  new_df = new_df.drop_duplicates()

  new_df.info()
  #print(new_df[['ICD9_CODE', 'EXPIRE_FLAG']])
  return new_df

  #valid_groups = [key for key, group in group_conditions.groups.keys() if (key[0], key[1], 1 - key[2]) in group_conditions.groups]
  #for key in valid_groups:
    #pass

In [ ]:
#balanced_data = split_set_half_expired(random_filtered_data)
balanced_data = random_filtered_data

Display notes

In [ ]:
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_rows', None)
#print(balanced_data['TEXT'].head(200))
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')

In [ ]:
print(balanced_data.columns)
print('\n'+"Number of rows: "+str(len(balanced_data)))

In [ ]:
balanced_data.info()

In [ ]:
no_duplicates = balanced_data.sort_values('ROW_ID').drop_duplicates(subset=['CGID', 'SUBJECT_ID'])  
no_duplicates = no_duplicates.drop(['TEXT'], axis=1)  # removes text notes
no_duplicates.info()

Save to CSV

In [ ]:
# path and name of the cvs file 
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_TSICU_COUNTS.csv'
#no_duplicates.to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

Gather- top ICD9, # of: patients, deceased, CGs, notes

In [ ]:
icd9_max_val = balanced_data['num_of_conditions'].max()
icd9_max_df = balanced_data[balanced_data['num_of_conditions'] == icd9_max_val]

print("Max ICD9 code: " + icd9_max_df['ICD9_CODE'].unique())
print("Number of patients: " + str(icd9_max_df['SUBJECT_ID'].nunique()))
print("Number of deceased patients: " + str((icd9_max_df.drop_duplicates(subset='SUBJECT_ID')['EXPIRE_FLAG'] == 1).sum()))
print("Number of caregivers: " + str(icd9_max_df['CGID'].nunique()))
unique_cgid = icd9_max_df['CGID'].unique()
unique_cgid_df = icd9_max_df[icd9_max_df['CGID'].isin(unique_cgid)].drop_duplicates(subset=['CGID', 'LABEL'])
unique_cgid_df['LABEL'] = unique_cgid_df['LABEL'].fillna('UNKNOWN')
unique_cgid_df['LABEL'] = unique_cgid_df['LABEL'].str.upper()
print("Caregiver labels: " + '\n' + str(unique_cgid_df['LABEL'].value_counts()))
print("Number of caregivers: " + str(icd9_max_df['CGID'].nunique()))
print("Number of notes: " + str(len(icd9_max_df)))

deceased patients for unfiltered set

In [ ]:
len(df_notes_TSICU['SUBJECT_ID'].unique())
deceased_TSICU = df_notes_TSICU[df_notes_TSICU['EXPIRE_FLAG'] == 1]
deceased_TSICU.info()

In [ ]:
sns.set()

gb_notes = deceased_TSICU.groupby(['CGID','SUBJECT_ID'])
num_notes = gb_notes['TEXT'].count()
print(num_notes)
print(f'number of notes: min={num_notes.min()}, max={num_notes.max()}, mean={num_notes.mean()},  median={num_notes.median()}')
sns.boxplot(x=num_notes)
sns.displot(num_notes[((num_notes>2) & (num_notes<15))])

Seperate deceased and living

In [ ]:
deceased_patients = balanced_data[balanced_data['EXPIRE_FLAG'] == 1]
deceased_patients.info()

In [ ]:
living_patients = balanced_data[balanced_data['EXPIRE_FLAG'] == 0]
living_patients.info()

Tokenized words

In [ ]:
#text = df_notes_TSICU['TEXT'].str.cat(sep=' ')
#words = nltk.word_tokenize(text)  # very slow

Count words in notes

In [ ]:
#stopwords = nltk.corpus.stopwords.words('english')
#words = [word.lower() for word in words if word.lower() not in stopwords and re.match(r'\b\w+\b', word)]  # also slow but only kinda slow
#word_counts = Counter(words)
#most_common_words = word_counts.most_common(300)

In [ ]:
#print("Rank\tWord\t\t\tCount")
#for rank, word_count in enumerate(most_common_words, start=1):
  #word, count = word_count
  #print(f"{rank}\t{word.ljust(20)}\t{count}")

Emotional words

In [ ]:
#emotional_words = []
#for word_count in most_common_words:
  #word = word_count[0]
  #synsets = wn.synsets(word)
  #if synsets:
    #senti_synset = swn.senti_synset(synsets[0].name())
    #if senti_synset.pos_score() > 0.45 or senti_synset.neg_score() > 0.45:
      #emotional_words.append(word)

#emotional_words

In [ ]:
#from textblob import TextBlob
#emotional_words = []
#for word_count in most_common_words:
  #word = word_count[0]
  #tb = TextBlob(word)
  #sentiment = tb.sentiment.polarity
  #if abs(sentiment) > 0.35:
    #emotional_words.append(word)

#emotional_words